<a href="https://colab.research.google.com/github/k248007-dot/Market-Basket-Analysis/blob/main/DS5001_Final_Project_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import collect_set
from pyspark.ml.fpm import FPGrowth

In [4]:
OnlineRetail = pd.read_csv('Online Retail.csv', encoding='ISO-8859-1')

In [26]:
OnlineRetail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [27]:
OnlineRetail.describe(include='object')

,InvoiceNo,StockCode,Description,InvoiceDate,Country
count,541909,541909,540455,541909,541909
unique,25900,4070,4223,23260,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,10/31/2011 14:41,United Kingdom
freq,1114,2313,2369,1114,495478


In [28]:
OnlineRetail.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [29]:
top_10_list = OnlineRetail['InvoiceNo'].value_counts().tail(10).index.tolist()
top_10_list

['536393',
 'C551348',
 '581491',
 '581487',
 'C581484',
 '581483',
 '581566',
 'C581499',
 'C536383',
 'C536379']

In [30]:
spark = SparkSession.builder \
    .appName("MarketBasketAnalysis") \
    .getOrCreate()

df = spark.read.csv(
    "Online Retail.csv",
    header=True,
    inferSchema=True
)

In [31]:
df_filtered = df.filter(
    (~col("InvoiceNo").startswith("C")) &
    (col("Quantity") > 0) &
    (col("CustomerID").isNotNull())
)

In [32]:
row_count_df = df.count()
row_count_df_filtered = df_filtered.count()
rows_diff = row_count_df - row_count_df_filtered
print(f"Number of rows: {row_count_df}")
print(f"Number of rows: {row_count_df_filtered}")
print(f"Rows excluded: {rows_diff}")

Number of rows: 541909
Number of rows: 397924
Rows excluded: 143985


In [33]:
basket_df = df_filtered.groupBy("InvoiceNo") \
    .agg(collect_set("Description").alias("items"))

In [34]:
basket_df

DataFrame[InvoiceNo: string, items: array<string>]

In [35]:
fp = FPGrowth(
    itemsCol="items",
    minSupport=0.02,
    minConfidence=0.3
)

model = fp.fit(basket_df)

In [36]:
frequent_itemsets = model.freqItemsets
frequent_itemsets.orderBy(col("freq").desc()).show(10)

+--------------------+----+
|               items|freq|
+--------------------+----+
|[WHITE HANGING HE...|1971|
|[REGENCY CAKESTAN...|1704|
|[JUMBO BAG RED RE...|1600|
|     [PARTY BUNTING]|1380|
|[ASSORTED COLOUR ...|1375|
|[LUNCH BAG RED RE...|1288|
|[SET OF 3 CAKE TI...|1146|
|           [POSTAGE]|1099|
|[LUNCH BAG  BLACK...|1052|
|[PACK OF 72 RETRO...|1029|
+--------------------+----+
only showing top 10 rows


In [37]:
rules = model.associationRules
rules.orderBy(col("lift").desc()).show(10)

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+--------------------+--------------------+------------------+------------------+--------------------+
|[GREEN REGENCY TE...|[PINK REGENCY TEA...|0.7208872458410351|24.033032354153647|0.021040138109624514|
|[PINK REGENCY TEA...|[GREEN REGENCY TE...|0.8944954128440367|23.994742362484896|0.021040138109624514|
|[PINK REGENCY TEA...|[GREEN REGENCY TE...|0.8273381294964028| 22.19325552582536|0.024816573154941735|
|[GREEN REGENCY TE...|[PINK REGENCY TEA...|0.6657018813314037| 22.19325552582536|0.024816573154941735|
|[PINK REGENCY TEA...|[ROSES REGENCY TE...|0.8478260869565217| 20.07063135099117|0.021040138109624514|
|[ROSES REGENCY TE...|[PINK REGENCY TEA...|0.5568326947637292|18.563760485864183| 0.02352179542511869|
|[PINK REGENCY TEA...|[ROSES REGENCY TE...|0.7841726618705036|18.56376048

In [ ]:
rules_pd = rules.toPandas()
rules_pd.to_csv("association_rules.csv", index=False)